<a href="https://colab.research.google.com/github/alinelena/janus-lavello/blob/main/single_point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install dependencies, pay special atention to install ase by hand

# Single point calculations with machine learnt interatomic potentials

janus-core contains various machine learnt interatomic potentials(MLIP), mace based (mace_mp, mace_off), chgnet, and matgl based m3gnet. Other will be added as their utility is proven beyond a specific material.

## Setup envitonment (Optional)

this step is to be done only if you want to pull latest versions of various component, add your own components of use a generic jupyter environment without dependencies preinstalled. The instructions are for googlecolab but may work on other systems too.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python3 -m pip install -U git+https://github.com/stfc/janus-core.git
!python3 -m pip install -U git+https://gitlab.com/ase/ase.git
!python3 -m pip install matgl
!python3 -m pip install chgnet

## Single point periodic

In [ ]:
from ase.build import bulk
from ase.io import write
from torch import set_default_dtype, float32

from janus_core.calculations.single_point import SinglePoint
NaCl = bulk('NaCl', 'rocksalt', a=5.63, cubic=True)
sp = SinglePoint(struct=NaCl,
                 architecture="mace_mp",
                 device='cpu',
                 calc_kwargs={'model_paths':'small','default_dtype':'float64'})

res_mace = sp.run()

# be sure you use single precision... chgnet default wants that
set_default_dtype(float32)
NaCl = bulk('NaCl', 'rocksalt', a=5.63, cubic=True)
sp = SinglePoint(struct=NaCl,
                 architecture="m3gnet",
                 device="cpu")

res_chgnet = sp.run()


# be sure you use single precision... chgnet default wants that
set_default_dtype(float32)
NaCl = bulk('NaCl', 'rocksalt', a=5.63, cubic=True)
sp = SinglePoint(struct=NaCl,
                 architecture="chgnet",
                 device="cpu")

res_m3gnet = sp.run()
print(f"  MACE[eV]: {res_mace['energy']}")
print(f"M3GNET[eV]: {res_m3gnet['energy']}")
print(f"CHGNET[eV]: {res_chgnet['energy']}")


## Simple Molecules

In [ ]:
from ase.build import molecule
from ase.io import write

from janus_core.calculations.single_point import SinglePoint

sp = SinglePoint(struct=molecule('H2O'),
                 architecture="mace_off",
                 device='cpu',
                 calc_kwargs={'model_paths':'medium'})

res = sp.run()
print(res)

## Sugar on salt

In [ ]:
from ase.build import surface,bulk, add_adsorbate, molecule
from ase.visualize import view
from ase.io import read,write

a = 5.63
NaCl = bulk('NaCl', crystalstructure="rocksalt",cubic=True,orthorhombic=True,a=5.63)*(6,6,3)
NaCl.center(vacuum=20.0, axis=2)
# get it from here https://raw.githubusercontent.com/stfc/janus-tutorials/main/data/sucrose.xyz
sugar = read("data/sucrose.xyz")
add_adsorbate(slab=NaCl, adsorbate=sugar, height=4.5, position=(10,10))
write("slab.xyz",NaCl)
sp = SinglePoint(struct_path="slab.xyz",
                 architecture="mace_mp",
                 device='cpu',
                 calc_kwargs={'model_paths':'small'})
res = sp.run()
print(res)
view(NaCl,viewer='x3d')